In [1]:
import numpy as np
import copy as copy
import matplotlib.pyplot as plt
from scipy.linalg import norm
import math 
import matplotlib.gridspec as gridspec
from scipy.interpolate import interp2d

f1=open("xc.dat",'r')
f2=open("yc.dat")
f3=open("u.dat")
f4=open("v.dat")



xc=f1.readlines()
xc = list(map(float, xc))
xc=np.array(xc)
#print(xc)
nim1=len(xc)
ni=nim1+1

yc=f2.readlines()
yc = list(map(float, yc))
yc=np.array(yc)
njm1=len(yc) #26
nj=njm1+1 #27

u1=f3.readlines()
u1 = list(map(float, u1))
u2d=np.reshape(u1,(ni,nj)) #27*27
u2d=np.array(u2d)
u2d=u2d.transpose()

v1=f4.readlines()
v1 = list(map(float, v1))
v2d=np.reshape(v1,(ni,nj)) #27*27
v2d=np.array(v2d)
v2d=v2d.transpose()
#print(v2d.shape)


#  xc and yc are the coordinates of the grid

#        o---------------o  yc(j)
#        |               |
#        |               |
#        |       P       |
#        |               |
#        |               |
#        o---------------o  yc(j-1)

#      xc(i-1)          xc(i)          

# The cell-node (P) has the coordinates xp(i), yp(j)

# compute the x-coordinates of the cell centres
xp= np.ones(nim1+1)
for i in range(1,nim1):
    xp[i]=0.5*(xc[i]+xc[i-1])
xp[0]=xc[0]
xp[-1]=xc[-1]

yp= np.ones(njm1+1)
for i in range(1,njm1):
    yp[i]=0.5*(yc[i]+yc[i-1])
yp[0]=yc[0]
yp[-1]=yc[-1]
#u for the faces:..............
u=np.zeros((nim1,njm1),dtype=np.float32)
v=np.zeros((nim1,njm1),dtype=np.float32)
for i in range(1,nim1):
    for j in range(1,njm1):
        u[i][j]=0.5*(u2d[i][j]+u2d[i][j+1])
        u[0][j]=u2d[0][j]
        u[i][0]=u2d[i][0]
for i in range(1,nim1):
    for j in range(1,njm1):
        v[i][j]=0.5*(v2d[i][j]+v2d[i][j+1])
        v[0][j]=v2d[0][j]
        v[i][0]=v2d[i][0]
#calculation for the distance between two cell center 
d_x=np.zeros(nim1)
d_y=np.zeros(njm1)
for i in range(0,nim1):
    d_x[i]= xp[i+1]-xp[i]
for i in range(0,njm1):
    d_y[i]= yp[i+1]-yp[i]
#calculation for del_x and del_y
del_x=np.zeros(nim1-1)
del_y=np.zeros(njm1-1)
for i in range(0,nim1-1):
    del_x[i]= xc[i+1]-xc[i]
for i in range(0,njm1-1):
    del_y[i]= yc[i+1]-yc[i]
#.........center dim of the domain for plot
cx=np.delete(xp,0)
cx = np.delete(cx,-1)
cy=np.delete(yp,0)
cy = np.delete(cy,-1)        
#............................................
Ta=20
T2=10
gamma=1/500 #k_by_cp
rho=1
#.................................
key=0
epc=10**(-5)
T_0=np.ones((len(del_x),len(del_y)),dtype=np.float32)
T=np.ones((len(del_x),len(del_y)),dtype=np.float32)
residu=[]
itaration=[]
count=0
#.............................................
while(key==0):
    for i in range(0,len(del_y)): #till 25 1.9
        for j in range(0,len(del_x)):            
            Fw=rho*u[i][j]*(del_y[i]);  Dw=(gamma*del_y[i])/d_x[j]
            Fe=rho*u[i][j+1]*(del_y[i]);De=(gamma*del_y[i])/d_x[j+1]
            Fs=rho*v[i][j]*(del_x[j]);  Ds=(gamma*del_x[j])/d_y[i]
            Fn=rho*v[i+1][j]*(del_x[j]);Dn=(gamma*del_x[j])/d_y[i+1]
            if(i==0 and j==0): #1
                Aw=0;As=0
                Ae=max(-Fe,(De-Fe/2),0);An=max(-Fn,(Dn-Fn/2),0)
                Su=Fw*T_0[i][j]+Fs*T_0[i][j]
                Sp=0
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Ae*T[i][j+1]+An*T[i+1][j]+Su)/Ap
                
   
            elif(i==0 and j==len(del_x)-1):#3 
                Aw=max(Fw,(Dw+Fw/2),0);As=0
                Ae=0;An=max(-Fn,(Dn-Fn/2),0)
                Su=Fs*T_0[i][j]-Fe*T_0[i][j]
                Sp=0
                Ap=Aw+Ae+An+As+(Fe+Fn-Fw-Fs)-Sp
                T[i][j]=(Aw*T[i][j-1]+An*T[i+1][j]+Su)/Ap
                #print('at 3',T_0[i][j])
            elif(i==len(del_y)-1 and j==len(del_x)-1):#9
                Aw=max(Fw,(Dw+Fw/2),0);As=max(Fs,(Ds+Fs/2),0)
                Ae=0;An=0
                Su=+De*T2-Fn*T_0[i][j]-Fe*T2
                Sp=-De
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Aw*T[i][j-1]+As*T[i-1][j]+Su)/Ap
            elif(i==len(del_y)-1 and j==0):#7 temp neumann 
                Aw=0;As=max(Fs,(Ds+Fs/2),0)
                Ae=max(-Fe,(De-Fe/2),0);An=0
                Su=-Fn*T_0[i][j]+Dw*Ta+Fw*Ta
                Sp=-Dw
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Ae*T[i][j+1]+As*T[i-1][j]+Su)/Ap
                print(T[i][j])
            elif(0<j<len(del_x)-1 and i==0):   #2 temp neumann 
                Aw=max(Fw,(Dw+Fw/2),0);As=0
                Ae=max(-Fe,(De-Fe/2),0);An=max(-Fn,(Dn-Fn/2),0)
                Su=Fs*T_0[i][j]
                Sp=0
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Ae*T[i][j+1]+Aw*T[i][j-1]+An*T[i+1][j]+Su)/Ap
                
            elif(j==len(del_x)-1 and 0<i<len(del_y)-1):#6 temp T1=10
                Aw=max(Fw,(Dw+Fw/2),0);As=max(Fs,(Ds+Fs/2),0)
                Ae=0;An=max(-Fn,(Dn-Fn/2),0)
                if(yc[i]<=0.136):
                    Su=-Fe*T_0[i][j]
                    Sp=0
                    Ap=Aw+Ae+An+As-Sp
                    T[i][j]=(An*T[i+1][j]+As*T[i-1][j]+Aw*T[i][j-1]+Su)/Ap
                else:
                    Su=+De*T2-Fe*T2
                    Sp=-De
                    Ap=Aw+Ae+An+As-Sp
                    T[i][j]=(An*T[i+1][j]+As*T[i-1][j]+Aw*T[i][j-1]+Su)/Ap
            elif(0<j<len(del_x)-1 and i==len(del_y)-1):#8
                Aw=max(Fw,(Dw+Fw/2),0);As=max(Fs,(Ds+Fs/2),0)
                Ae=max(-Fe,(De-Fe/2),0);An=0
                Su=-Fn*T_0[i][j]
                Sp=0
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Ae*T[i][j+1]+Aw*T[i][j-1]+As*T[i-1][j]+Su)/Ap
            elif(0<i<len(del_y)-1 and j==0):#4
                Aw=0;As=max(Fs,(Ds+Fs/2),0)
                Ae=max(-Fe,(De-Fe/2),0);An=max(-Fn,(Dn-Fn/2),0)
                if(yc[i]>=1.864):#hc=0.136 temp 20
                    Su=+Dw*Ta+Fw*Ta
                    Sp=-Dw
                    Ap=Aw+Ae+An+As-Sp
                    T[i][j]=(Ae*T[i][j+1]+As*T[i-1][j]+An*T[i+1][j]+Su)/Ap
                    #print('at 4',T[i][j])
                else: #temp neumann 
                    Su=Fw*T_0[i][j]
                    Sp=0
                    Ap=Aw+Ae+An+As-Sp
                    T[i][j]=(Ae*T[i][j+1]+As*T[i-1][j]+An*T[i+1][j]+Su)/Ap
            else:
                Aw=max(Fw,(Dw+Fw/2),0);As=max(Fs,(Ds+Fs/2),0)
                Ae=max(-Fe,(De-Fe/2),0);An=max(-Fn,(Dn-Fn/2),0)
                Su=0
                Sp=0
                Ap=Aw+Ae+An+As-Sp
                T[i][j]=(Ae*T[i][j+1]+Aw*T[i][j-1]+An*T[i+1][j]+As*T[i-1][j]+Su)/Ap
    count=count+1
    itaration.append(count)            #itaration[j]=count
    #R[j]=(norm(T-T_0))/len(T)
    R=(norm(T-T_0))/len(T)
    residu.append(R)
    if(R<epc):
        key=1
    T_0=copy.copy(T)
    
#print(T)
[X,Y]=np.meshgrid(cx,cy) #to view temp plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(X, Y, T)
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Temp Plot')
ax.set_xlabel('x (cm)')
ax.set_ylabel('y (cm)')
plt.savefig("output2.png",dpi=600)
plt.show()

# plt.quiver(xc,yc,u,v,9)
# plt.savefig("output2.png",dpi=600)
# plt.show()

82.76476
88.13613
88.57181
88.60821
88.61143
88.611786
88.61187
88.611916
88.61195
88.61198
88.612
88.612015
88.61203
88.61204
88.612045
88.61205
88.61206
88.61206
88.61206
88.61207
88.61207
88.61207
88.61207
88.61207
88.612076
88.612076
88.612076
88.612076
88.612076
88.612076
88.612076
88.612076
88.61208
88.61208
88.61208
88.61209
88.6121
88.61211
88.612114
88.61212
88.61213
88.612144
88.61215
88.61217
88.612175
88.61219
88.612206
88.61221
88.61223
88.61224
88.61226
88.612274
88.61229
88.612305
88.61232
88.612335
88.61235
88.612366
88.61238
88.6124
88.61242
88.612434
88.61245
88.612465
88.61249
88.6125
88.61252
88.61253
88.61256
88.61257
88.61259
88.6126
88.612625
88.61264
88.612656
88.61268
88.61269
88.61271
88.612724
88.61275
88.61276
88.61278
88.61279
88.61281
88.61283
88.61285
88.61286
88.61288
88.61289
88.61291
88.61292
88.61294
88.61295
88.61297
88.61298
88.613
88.613014
88.61303
88.613045
88.61305
88.61307
88.61308
88.6131
88.61311
88.61312
88.61314
88.613144
88.61316
88.613174

88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815
88.613815


KeyboardInterrupt: 

In [ ]:
print(u.shape)

In [4]:
print(yp.shape)

(27,)


In [2]:
del_x.shape

(25,)

In [ ]:
u[25][0]

In [ ]:
print(xp.shape)
print(cx.shape)


In [ ]:
print(yc)